# Global

In [ ]:
model_saved_name="model_weights.ckpt"
data_dir=r'C:\Users\ossia\Documents\GitHub\MagLearn-Bristol-2\Single Pipeline\preprocessed_training_dataset_3'

## processing...

In [ ]:
import numpy as np
import torch
import os
import torch.optim as optim
import torch.nn as nn
import matplotlib.pyplot as plt
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

import matplotlib.pyplot as plt

import NW_LSTM
import NN_DataLoader

# Check if CUDA is available and if so, set the device to GPU
device = torch.device("cpu")
print("Device using ", device)

class MyDataset(Dataset):

    def __init__(self, x_data, y_data):
        self.x_data = torch.tensor(x_data, dtype=torch.float32)
        self.y_data = torch.tensor(y_data, dtype=torch.float32)

    def __len__(self):
        return len(self.x_data)

    def __getitem__(self, idx):
        return self.x_data[idx], self.y_data[idx]



In [ ]:
# Initialize the list of materials to validate
materials = []

# Check if data_dir is a real directory
if os.path.exists(data_dir) and os.path.isdir(data_dir):
    # Get the list of materials in the data_dir folder
    subfolders = [item for item in os.listdir(data_dir)
                  if os.path.isdir(os.path.join(data_dir, item))]

    # Check if there are any subfolders
    if subfolders:
        # Save the names of the subfolders to the materials list
        materials = subfolders
    else:
        print("No material datasets found in directory specified by data_dir, preprocessed (normalised, down sampled and split) training materials should be saved into subfolders within this directory, where each subfolder is the name of a training material.")
else:
    print("Directory specified by data_dir does not seem to exist, please ensure data_dir specifies a folder containing subfolders of processed training datasets for each material.")

# Print the list of training materials if they exist
if materials:
    print("Identified training data for materials:", materials)

In [ ]:
import Maglib

for mat in materials:
    material_dir = os.path.join(data_dir, mat)
    magData = Maglib.MagLoader(os.path.join(material_dir, 'test.mat'))

    # Instantiate the model with appropriate dimensions
    model = NW_LSTM.get_global_model().to(device)
    model.load_state_dict(torch.load(os.path.join(material_dir, model_saved_name), map_location=device)) # Load trained material model from .ckpt file

    x_data = np.zeros([magData.b.shape[0], magData.b.shape[1], 3])
    x_data[:, :, 0] = magData.b
    x_data[:, :, 1] = magData.freq
    x_data[:, :, 2] = magData.temp

    idx = 0
    dataNums = magData.freq.shape[0]
    # no more than 2000
    if(dataNums>2000):dataNums=2000


    x_data = x_data[idx:idx + dataNums, :, :]
    y_data = magData.loss[idx:idx + dataNums, :]

    # Now we can pass a batch of sequences through the model
    inputs = torch.tensor(x_data, dtype=torch.float32)

    outputs = model(inputs)

    total_params = sum(p.numel() for p in model.parameters())

    print('Data size ', magData.b.shape[0])
    print('model parameters: ', total_params)

    # get model performance

    # get loss
    pred = outputs.detach().numpy()
    real = y_data

    import linear_std
    import MagNet

    std_loss = linear_std.linear_std()
    std_loss.load(os.path.join(material_dir, "std_loss.stdd"))

    pred = std_loss.unstd(pred)
    real = std_loss.unstd(real)

    relv_error = abs(pred - real) / real # Relative absolute error
    mean_relv_error = np.mean(relv_error, axis=0)
    MagNet.Mag_plot(mat, relv_error, save_path = data_dir)



In [ ]:
if(0):
    bad_example=[]
    good_example=[]
    for idx in range(error.shape[0]):
        if abs(error[idx])>0.2:
            bad_example.append(idx)
        elif abs(error[idx])<0.04:
            good_example.append(idx)


    std_temp= linear_std.linear_std()
    std_temp.load(data_dir+r"\std_temp.stdd")

    std_freq= linear_std.linear_std()
    std_freq.load(dataset_path+r"\std_freq.stdd")


    bad_freq=[]
    bad_temp=[]

    good_freq=[]
    good_temp=[]

    for idx in bad_example:
        bad_freq.append(std_freq.unstd(magData.freq[idx]))
        bad_temp.append(std_temp.unstd(magData.temp[idx]))

    for idx in good_example:
        good_freq.append(std_freq.unstd(magData.freq[idx]))
        good_temp.append(std_temp.unstd(magData.temp[idx]))


    # plot the distribution of good_freq vs bad_freq
    plt.figure()
    plt.title("error distribution in freq")
    plt.hist(np.array(good_freq),label="good")
    plt.hist(np.array(bad_freq),label="bad")
    plt.legend()
    plt.show()

    # plot the distribution of good_temp vs bad_temp
    plt.figure()
    plt.title("error distribution in temp")
    plt.hist(np.array(good_temp),label="good")
    plt.hist(np.array(bad_temp),label="bad")
    plt.legend()
    plt.show()

    # plot the distribution of error
    drawing=False

    if drawing:
        for idx in bad_example:
            plt.figure()
            # using title to show the temp and freq
            plt.title('temp: ' + str(std_temp.unstd(magData.temp[idx])) +
                    '\nfreq: ' + str(std_freq.unstd(magData.freq[idx])))

            plt.plot(magData.b[idx,:])
            plt.show()